In [1]:
from UNet import *
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd

### load data

In [2]:
raw_image = dd.io.load('4channel_vf.h5')
vector_field = dd.io.load('vector_fields.h5')

In [3]:
raw_image = raw_image.astype(np.float)

In [4]:
vector_field.shape

(105, 4, 840, 840)

### set hyperparameters

In [5]:
batch_size = 1
img_size = raw_image.shape[-1]
lr = .0002
epoch = 1

### transform to torch tensor

In [6]:
tensor_x = torch.stack([torch.Tensor(i) for i in raw_image])
tensor_y = torch.stack([torch.Tensor(i) for i in vector_field])

### create dataset

In [7]:
my_dataset = data.TensorDataset(tensor_x, tensor_y)

### input pipeline

In [8]:
img_batch = data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

### initiate U-Net model generator

In [9]:
generator = UnetGenerator(1, 4, 64)


------Initiating the U-Net------



### set loss function and optimizer

In [10]:
recon_loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

### training the model

In [11]:
for i in range(epoch):
    for k, (img, vec_field) in enumerate(img_batch):
        gen_optimizer.zero_grad()
        x = Variable(img)#.cuda(0)
        y_ = Variable(vec_field)#.cuda(0)
        print("finish y_")
        y = generator.forward(x)
        loss = recon_loss_func(y, y_)
        print(loss)
        loss.backward()
        gen_optimizer.step()
        break

OSError: [Errno 12] Cannot allocate memory